<a href="https://colab.research.google.com/github/rituparrna33/Market-Basket-Analysis/blob/main/MBA_using_Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Apriori Algorithm for Market Basket Analysis

In [2]:
#Dataset: https://archive.ics.uci.edu/dataset/352/online+retail

# Further Readings :
#     https://www.ibm.com/think/topics/apriori-algorithm

In [3]:
import pandas as pd
import numpy as np

In [4]:
data=pd.read_excel("Online Retail 2.xlsx")
data.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
data['Description'].nunique()

4223

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [7]:
data.shape

(541909, 8)

In [8]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], format="%Y %m %d %H:%M")

In [9]:
data["Year/Month"] = data["InvoiceDate"].dt.to_period("M")

In [10]:
data['UnitPrice'].sample(5)
mask = data['UnitPrice'].astype(str).str.contains(',')
data[mask]


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Year/Month


In [11]:
data['UnitPrice'].astype(str).str.contains(',').sum()

np.int64(0)

In [12]:
missing=data.isnull().sum()
missing

,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0
Year/Month,0


In [13]:
data = data[(data["Quantity"] > 0) & (data["UnitPrice"] > 0)]

In [14]:
data = data[data["Description"].notnull()]

In [15]:
data = data.fillna('#NV')

In [16]:
data["TotalPrice"] = data["Quantity"] * data["UnitPrice"]

In [17]:
data = data[data['Country'] == 'France']

In [18]:
data_processed = data[['InvoiceNo', 'Description']]
data_encoded = pd.get_dummies(data_processed, columns=['Description'])
data_encoded.columns = data_encoded.columns.str.replace("Description_", "")
basket = data_encoded.groupby('InvoiceNo').sum()

basket.head()

,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
basket[basket > 0] = 1

In [22]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(basket,min_support=0.02,use_colnames=True)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [23]:
rules_confidenz = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)
print(rules_confidenz.shape)
rules_c = rules_confidenz.round(3)
rules_c.head()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

(2405, 14)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(DOLLY GIRL BABY GIFT SET),( SPACEBOY BABY GIFT SET),0.023,0.041,0.023,1.000,24.500,1.0,0.022,inf,0.982,0.562,1.000,0.781
1,(10 COLOUR SPACEBOY PEN),(POSTAGE),0.031,0.765,0.031,1.000,1.307,1.0,0.007,inf,0.242,0.040,1.000,0.520
2,(3 PIECE SPACEBOY COOKIE CUTTER SET),(POSTAGE),0.036,0.765,0.033,0.929,1.213,1.0,0.006,3.286,0.182,0.043,0.696,0.486
3,(36 PENCILS TUBE WOODLAND),(36 PENCILS TUBE RED RETROSPOT),0.023,0.046,0.020,0.889,19.358,1.0,0.019,8.587,0.971,0.421,0.884,0.667
4,(60 CAKE CASES DOLLY GIRL DESIGN),(POSTAGE),0.023,0.765,0.023,1.000,1.307,1.0,0.005,inf,0.240,0.030,1.000,0.515


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


### Insights from the Confidence Metric

The **confidence** metric for a rule {A} -> {B} is calculated as the support of the union of A and B divided by the support of A. In simpler terms, it tells us the probability of purchasing item(s) B, given that item(s) A have already been purchased.

**How to get insights from Confidence:**

*   **Identify reliable rules:** Rules with high confidence values are more reliable predictors of co-occurrence. If the confidence of {A} -> {B} is high, it means that when a customer buys A, they are very likely to also buy B.
*   **Make recommendations:** High-confidence rules can be used to make product recommendations. If a customer has item A in their basket, the system can recommend item B.
*   **Cross-selling strategies:** Businesses can use high-confidence rules to identify products that are frequently bought together and develop cross-selling strategies, such as placing these items near each other in a store or online.
*   **Understand customer behavior:** Analyzing rules with varying confidence levels can provide insights into different aspects of customer purchasing behavior.
*   **Set thresholds:** You can set a minimum confidence threshold to filter the association rules and focus on the most significant relationships.

**Example:**

If the rule `{Laptop}` -> `{Laptop Bag}` has a confidence of 0.90, it means that 90% of customers who buy a laptop also buy a laptop bag. This is a strong association that can be used for effective recommendations and store layout optimization.

In [24]:
rules_lift = association_rules(frequent_itemsets, metric="lift", min_threshold= 2.5)
print(rules_lift.shape)
rules_l = rules_lift.round(3)
rules_l.head()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

(10480, 14)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,( SPACEBOY BABY GIFT SET),(DOLLY GIRL BABY GIFT SET),0.041,0.023,0.023,0.563,24.500,1.0,0.022,2.233,1.000,0.562,0.552,0.781
1,(DOLLY GIRL BABY GIFT SET),( SPACEBOY BABY GIFT SET),0.023,0.041,0.023,1.000,24.500,1.0,0.022,inf,0.982,0.562,1.000,0.781
2,(36 PENCILS TUBE RED RETROSPOT),(36 PENCILS TUBE WOODLAND),0.046,0.023,0.020,0.444,19.358,1.0,0.019,1.759,0.994,0.421,0.431,0.667
3,(36 PENCILS TUBE WOODLAND),(36 PENCILS TUBE RED RETROSPOT),0.023,0.046,0.020,0.889,19.358,1.0,0.019,8.587,0.971,0.421,0.884,0.667
4,(36 PENCILS TUBE RED RETROSPOT),(PLASTERS IN TIN WOODLAND ANIMALS),0.046,0.171,0.020,0.444,2.600,1.0,0.013,1.492,0.645,0.104,0.330,0.282


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Step 2: Insights
1️⃣ Strong mutual association between gift sets

SPACEBOY BABY GIFT SET → DOLLY GIRL BABY GIFT SET

Confidence = 0.563 → 56% of buyers of SPACEBOY also buy DOLLY GIRL.

Lift = 24.5 → these products appear together much more than random chance.

Action: Consider bundling these two gift sets, offering a combo discount, or “frequently bought together” suggestions.

2️⃣ Pencils are often bought in pairs

36 PENCILS TUBE RED RETROSPOT → 36 PENCILS TUBE WOODLAND

Confidence = 0.444 → nearly half of the buyers of RED RETROSPOT pencils also buy WOODLAND.

Lift = 19.36 → very strong association, much higher than random.

Action: Can promote as a set of complementary pencil tubes, or show as recommendations in the cart.

3️⃣ Cross-category suggestions

36 PENCILS TUBE RED RETROSPOT → PLASTERS IN TIN WOODLAND ANIMALS

Confidence = 0.444, Lift = 2.6 → lower than pencil-pairing but still meaningful.

Action: Consider cross-sell or bundle small stationery items together.

4️⃣ Bidirectional rules

Some rules have flipped antecedent/consequent:

36 PENCILS TUBE WOODLAND → 36 PENCILS TUBE RED RETROSPOT has confidence = 0.889.

Insight: These high-lift bidirectional associations show strong complementary purchases, ideal for recommendation engines.

Step 3: Actionable Recommendations

Bundle high-lift items:

Baby gift sets and pencils sets → combo offers or curated product packs.

Cart recommendations:

Add “Customers who bought this also bought…” suggestions using these rules.

Promotional campaigns:

Targeted marketing emails for complementary products (e.g., if someone buys RED RETROSPOT pencils, suggest WOODLAND pencils or plasters).

Stock planning:

Ensure high-lift products are available together — reduces missed cross-sell opportunities.

In [25]:
rules_support = association_rules(frequent_itemsets, metric="support", min_threshold=0.03)
print(rules_support.shape)
rules_s = rules_support.round(3)
rules_s.head()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

(2860, 14)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(10 COLOUR SPACEBOY PEN),(POSTAGE),0.031,0.765,0.031,1.000,1.307,1.0,0.007,inf,0.242,0.040,1.000,0.520
1,(POSTAGE),(10 COLOUR SPACEBOY PEN),0.765,0.031,0.031,0.040,1.307,1.0,0.007,1.010,1.000,0.040,0.010,0.520
2,(POSTAGE),(3 PIECE SPACEBOY COOKIE CUTTER SET),0.765,0.036,0.033,0.043,1.213,1.0,0.006,1.008,0.749,0.043,0.008,0.486
3,(3 PIECE SPACEBOY COOKIE CUTTER SET),(POSTAGE),0.036,0.765,0.033,0.929,1.213,1.0,0.006,3.286,0.182,0.043,0.696,0.486
4,(4 TRADITIONAL SPINNING TOPS),(POSTAGE),0.071,0.765,0.056,0.786,1.027,1.0,0.001,1.095,0.028,0.072,0.087,0.430


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Step 1: Understand the metrics

Support: Fraction of all transactions containing the itemset.

Example: 10 COLOUR SPACEBOY PEN + POSTAGE appears in 3.1% of all transactions.

Confidence: Probability that the consequent is purchased when the antecedent is purchased.

Example: If someone buys 10 COLOUR SPACEBOY PEN, there’s a 100% chance they also have POSTAGE.

Lift: How much more likely the consequent occurs with the antecedent than random chance.

1 means positive association, <1 means negative, =1 means independent.

Step 2: Insights from your rules

High confidence rules involving POSTAGE

10 COLOUR SPACEBOY PEN → POSTAGE has 100% confidence.

3 PIECE SPACEBOY COOKIE CUTTER SET → POSTAGE has 92.9% confidence.

4 TRADITIONAL SPINNING TOPS → POSTAGE has 78.6% confidence.

✅ Insight: Customers buying individual products almost always have POSTAGE included — expected, but confirms that shipping charges are consistently applied.

Asymmetric rules

POSTAGE → 10 COLOUR SPACEBOY PEN has 4% confidence.

Meaning: While buyers of the pen always pay for postage, buyers who just have POSTAGE don’t always buy this pen.

✅ Insight: Antecedent choice matters — POSTAGE itself is common, so don’t treat it as a product for cross-selling.

Lift values

Most lifts > 1 (1.027–1.307) → slight positive association.

Example: Buying 10 COLOUR SPACEBOY PEN increases the likelihood of POSTAGE by ~1.3x over random.

Lift > 1 confirms that these items are somewhat associated, but not extremely strong (lift < 2).

Product bundling opportunities

High-confidence rules like product → POSTAGE are expected, but if we remove POSTAGE, look for rules between products themselves for cross-selling.

In this snippet, POSTAGE dominates as a consequent, so consider filtering it out to find meaningful product-to-product associations.

Step 3: Recommendations / Actions

Bundle or cross-sell frequently bought items

Track product pairs with high confidence and lift (excluding POSTAGE) to create bundles or promotions.

Refine association rules

Exclude rules where POSTAGE is antecedent or consequent to focus on actual product relationships.

Promotions / upsell opportunities

For products that appear together often (high lift + confidence), offer bundle discounts, related product recommendations, or “frequently bought together” suggestions.